# Metody pozyskiwania wiedzy z danych - projekt zaliczeniowy

Narzędzie pozwalające użytkownikowi na estymację ceny mieszkania w zależności od podanych parametrów.

Skład zespołu: 
- Zapalski, 
- Szkatuła, 
- Pietrzyk, 
- Śmiałowski

## To do:
- opis zestawu danych i jego źródła
- analiza eksploracyjna zestawu danych (info o zmiennych, statystki opisowe, rozkłady, korelacje, .... )
- analiza wizualna (przedstawienie danych w formie wizualnej, przeprowadzenie analiz zależności pomiędzy zmiennymi, heatmapy z najczęstszymi miejscami, ... )
- ocztszczenie danych (outliery, NA itd. )
- pierwszy model
- feature enginering
- zastosowanie różnych typów modeli (?)
- hyperparameters tuning na najlepszym modelu
- stworzenie coś na wzór UX w noteboooku

## Notes:
- zrobić coś na wzór UI dla użytkownika końcowego (może być jako suwaczki w jupyterze albo API)
- lista dzielnic i miast, feature enginering na zmiennej adres tak zeby wyciagnac dzielnice
- analiza danych, statystki opisowe, wykresy i mapki

## Code

In [31]:
import pandas as pd
import matplotlib as plt
import numpy as np

from sklearn.metrics import mean_squared_error, r2_score

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

from sklearn.tree import DecisionTreeRegressor
import catboost as ctb

In [32]:
!pwd

/Users/mikolaj/Desktop/2 semestr/wiedza-dane/!projekt


In [33]:
df = pd.read_csv("data/Houses.csv", encoding='latin-1',index_col=0)
df.head()

,address,city,floor,id,latitude,longitude,price,rooms,sq,year
0,Podgórze Zab³ocie Stanis³awa Klimeckiego,Kraków,2.0,23918.0,50.049224,19.970379,749000.0,3.0,74.05,2021.0
1,Praga-Po³udnie Grochowska,Warszawa,3.0,17828.0,52.249775,21.106886,240548.0,1.0,24.38,2021.0
2,Krowodrza Czarnowiejska,Kraków,2.0,22784.0,50.066964,19.920025,427000.0,2.0,37.00,1970.0
3,Grunwald,Poznañ,2.0,4315.0,52.404212,16.882542,1290000.0,5.0,166.00,1935.0
4,Ochota Gotowy budynek. Stan deweloperski. Osta...,Warszawa,1.0,11770.0,52.212225,20.972630,996000.0,5.0,105.00,2020.0


# Feature engineering - dzielnice

In [34]:
df.sample(40)

,address,city,floor,id,latitude,longitude,price,rooms,sq,year
7872,Grzegórzki Kazimierza Kordylewskiego,Kraków,2.0,24700.0,50.060336,19.962316,629000.00,2.0,47.13,2009.0
5859,Wzgórza Krzes³awickie ul.Gustawa Morcinka,Kraków,0.0,26734.0,50.104781,20.036274,382168.00,3.0,61.64,2022.0
5173,Krowodrza ul. Pr¹dnicka,Kraków,2.0,18609.0,50.088279,19.938923,639000.00,2.0,57.70,1939.0
13163,Bia³o³êka,Warszawa,3.0,15706.0,52.319665,21.021177,342000.00,1.0,34.20,2004.0
18036,Podgórze Ignacego Krasickiego,Kraków,3.0,22049.0,50.039075,19.939064,389000.00,2.0,38.09,1970.0
2534,Rembertów Odbierz klucze szybko>> Wysoki stand...,Warszawa,5.0,8454.0,52.261415,21.162819,449901.00,4.0,66.07,2021.0
11621,Grunwald Jeleniogórska 1/3,Poznañ,4.0,5616.0,52.404212,16.882542,401354.00,2.0,54.98,2021.0
18339,Mokotów,Warszawa,4.0,15651.0,52.193987,21.045781,540000.00,2.0,50.00,1978.0
8786,Stare Miasto Pi¹tkowo ul. Królewska,Poznañ,2.0,1624.0,52.458891,16.900072,365000.00,2.0,44.10,2001.0
4517,Nowe Miasto Polanka 2,Poznañ,4.0,4704.0,52.399759,16.954660,710446.00,4.0,82.61,2020.0


Możemy zauważyć, że w kolumnie *address* każda obserwacja zaczyna się od nazwy dzielnicy.

To, że nazwy są źle zakodowane nas nie intereseuje, model i tak będzie widział to jako osobne liczby.

Problemem może okazać się rozróżnienie podobych nazw dzielnic np. Wola w Warszawie i Wola Duchacka w Krakowie oraz różna liczba słów w nazwie dzielnicy (Śródmieście - 1 , Stare Miasto - 2 itd. )

In [35]:
df["address"].value_counts()[:30]

Mokotów                                   426
Wola                                      384
Nowe Miasto Malta ul. Katowicka           377
ródmiecie                               352
Bia³o³êka                                 248
Mistrzejowice ul. Piasta Ko³odzieja       224
Mistrzejowice                             221
Praga-Po³udnie                            216
Bemowo                                    204
Stare Miasto Naramowice ul. Czarnucha     199
Praga-Po³udnie Goc³aw                     193
Podgórze                                  191
Nowe Miasto Rataje                        181
Grzegórzki                                178
Stare Miasto                              178
Ursynów                                   174
Pr¹dnik Bia³y                             174
Ochota                                    159
Nowa Huta Czy¿yny Nowohucka 46            142
Krowodrza                                 142
Podgórze ul. Stanis³awa Klimeckiego       138
Mokotów ul. Domaniewska 34B       

wyciągnięcie np. top20 pod względem częstości występowania nie rozwiąże problemu, nowe osiedla podawane z ulica (Mistrzejowice ul. Piasta Ko³odzieja) ze sporą ilością ofert zawyżają tą listę

Możliwe rozwiązania:

    -skonstruowanie słownika z nazwami dzielnic oraz poprawienie danych, tak aby wyświetlały się poprawnie (formatowanie
    -ewentualnie ręczne poprawienie każdego rekordu

In [36]:
wwa= ["Bemowo",
"Białołęka",
"Bielany",
"Mokotów",
"Ochota",
"Praga Południe",
"Praga Północ",
"Rembertów",
"Śródmieście",
"Targówek",
"Ursus",
"Ursynów",
"Wawer",
"Wesoła",
"Wilanów",
"Włochy",
"Wola",
"Żoliborz"]
wwa

['Bemowo',
 'Białołęka',
 'Bielany',
 'Mokotów',
 'Ochota',
 'Praga Południe',
 'Praga Północ',
 'Rembertów',
 'Śródmieście',
 'Targówek',
 'Ursus',
 'Ursynów',
 'Wawer',
 'Wesoła',
 'Wilanów',
 'Włochy',
 'Wola',
 'Żoliborz']

# Model

In [37]:
# wybranie zmiennych jakościowych
obj_feats = list(df.select_dtypes(np.object).columns)

# zamiana ich na liczby
for feat in obj_feats:
    df["{}_cat".format(feat)] = df[feat].factorize()[0]

black_list = ["address", "city"]

feats = df.select_dtypes(include=[np.number, np.bool]).columns
feats = [x for x in feats if x not in black_list]

X = df[feats].to_numpy()
y = df["price"].to_numpy()

# podział na train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [38]:
model = ctb.CatBoostRegressor(verbose=False)
#cross_val_score(model, X_train, y_train, cv=3, scoring='r2')

In [39]:
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [40]:
df["price"].median()

520000.0

In [41]:
print ( r2_score(y_test, y_pred), mean_squared_error(y_test, y_pred, squared=False) )

0.9586013646209987 104678.97093596107


# Explainer Dashboard

In [47]:
from explainerdashboard import ClassifierExplainer, ExplainerDashboard

ContextualVersionConflict: (Jinja2 2.11.2 (/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages), Requirement.parse('Jinja2>=3.0'), {'flask'})

In [42]:
## Scores

## Catboost - no parameters - 0.932 r2 - 147359 mse